In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io

In [2]:
import numpy as np
import os
from PIL import Image

In [3]:
# Define the data generator with the same parameters for images and masks
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect'
)

image_directory = 'preptest/image/'
mask_directory = 'preptest/mask/'
SIZE = 128


def load_images_from_folder(folder, size, is_mask=False):
    images = []
    for filename in os.listdir(folder):
        if (filename.endswith('.jpg') and not is_mask) or (filename.endswith('_segmentation.png') and is_mask):
            img_path = os.path.join(folder, filename)
            # print(f"Loading image: {img_path}")  # Debug statement
            img = io.imread(img_path)
            if img is not None:
                img = Image.fromarray(img)
                img = img.resize((size, size))
                images.append(np.array(img))
            else:
                print(f"Failed to load image: {img_path}")  # Debug statement
    return np.array(images)

# Load images and masks
images = load_images_from_folder(image_directory, SIZE)
masks = load_images_from_folder(mask_directory, SIZE, is_mask=True)

In [4]:
print(f"Number of images loaded: {len(images)}")
print(f"Number of masks loaded: {len(masks)}")

Number of images loaded: 5
Number of masks loaded: 5


In [5]:
# Ensure images and masks have the same shape and add a channel dimension if needed
if len(images.shape) == 3:
    images = np.expand_dims(images, axis=-1)
if len(masks.shape) == 3:
    masks = np.expand_dims(masks, axis=-1)

In [6]:
# Combine images and masks into one array for simultaneous augmentation
combined = np.concatenate((images, masks), axis=3)

In [10]:
# Function to split the combined array back into images and masks
def split_combined_array(combined_array):
    images = combined_array[:, :, :, :3]
    masks = combined_array[:, :, :, 3]
    return images, masks

In [12]:

# Create directories if they don't exist
os.makedirs('augmented/image', exist_ok=True)
os.makedirs('augmented/mask', exist_ok=True)

In [11]:
# Augment and save images and masks
i = 0
for batch in datagen.flow(combined, batch_size=16, save_to_dir='augmented', save_prefix='aug', save_format='png'):
    augmented_images, augmented_masks = split_combined_array(batch)
    for j in range(augmented_images.shape[0]):
        img = Image.fromarray(augmented_images[j].astype('uint8'), 'RGB')
        mask = Image.fromarray(augmented_masks[j].astype('uint8'), 'L')  # 'L' mode for grayscale masks
        img.save(f'augmented/image/aug_{i}_{j}.png')
        mask.save(f'augmented/mask/aug_{i}_{j}.png')
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Major Project\\augmented\\image\\aug_0_0.png'